In [2]:
import json

import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('data/2ch_posts.csv')

In [46]:
system = f"""
    You are given exactly one post from a special forum where people post their problems and ask for advice.
    The forum is called "2ch" and it is very popular in Russia.
    Your task is to determine if this message is *toxic* or not.
    Respond with exactly one number: '1' if the message is toxic and '0' otherwise. Don't write anything else.
"""


mapping = []
for post in tqdm(df['text']):  
    for _ in range(5):
        r = requests.post(url="http://localhost:11434/api/generate",
                          json={
                              "model": "llama2-uncensored",
                              "prompt": post,
                              "stream": False,
                              # "context": context,
                              "system": system,
                          })
        if r.status_code == 200:
            response_str = r.content.decode('utf-8')
            response = json.loads(response_str)["response"]
            spl = response.strip().replace(',', ' ').replace('  ', ' ').replace('\n', ' ').strip()

            if spl != '1' and spl != '0':
                # print(response)
                continue
            mapping.append((post, int(spl)))
            # print(post, ": ", spl, sep='')
            break
        else:
            print("Response status code is {r.status_code}")
            continue


100%|██████████| 13344/13344 [11:06:14<00:00,  3.00s/it]  


In [47]:
df = pd.DataFrame(mapping, columns=['text', 'toxic'])

In [48]:
df.to_csv('data/2ch_labeled_llama.csv', index=True, index_label='id')